In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 기본 URL 설정
base_url = "https://www.youthcenter.go.kr"
list_url = f"{base_url}/youngPlcyUnif/youngPlcyUnifList.do"

# 크롤링한 데이터 저장할 리스트
all_data = []

# 1. 페이지 반복 요청 (목록 페이지)
for page in range(1):  # 페이지 수 조정 (1~10페이지 크롤링)
    print(f"Processing page {page}...")
    response = requests.get(
        list_url,
        params={"pageIndex": page, "pageUnit": 60}  # 페이지 및 항목 수 설정
    )
    soup = BeautifulSoup(response.text, 'html.parser')

    # 2. 조건에 맞는 링크 수집
    links = []
    items = soup.find_all('li')
    for item in items:
        badge = item.find('div', class_='badge')
        if badge and ("상시" in badge.text or "진행중" in badge.text):
            link_tag = item.find('a', class_='tit')
            if link_tag:
                onclick_data = link_tag.get('onclick')
                if onclick_data:
                    biz_id = onclick_data.split("'")[1]  # 'f_Detail' 내부의 ID 추출
                    detail_link = f"{base_url}/youngPlcyUnif/youngPlcyUnifDtl.do?bizId={biz_id}"
                    links.append(detail_link)

    # 3. 각 세부 페이지 데이터 크롤링
    for link in links:
        try:
            detail_response = requests.get(link)
            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

            # 제목 및 설명
            title = detail_soup.find('h2', class_='doc_tit01 type2').text.strip() if detail_soup.find('h2', class_='doc_tit01 type2') else None
            description = detail_soup.find('p', class_='doc_desc').text.strip() if detail_soup.find('p', class_='doc_desc') else None

            # 정책 요약 정보 크롤링
            summary_data = []
            tables = detail_soup.find_all('div', class_='table_wrap')
            for table in tables:
                rows = table.find_all('li')
                for row in rows:
                    title_tag = row.find('div', class_='list_tit')
                    content_tag = row.find('div', class_='list_cont')
                    if title_tag and content_tag:
                        summary_data.append({
                            "title": title_tag.text.strip(),
                            "content": content_tag.text.strip()
                        })

            # 결과 저장
            all_data.append({
                "link": link,
                "title": title,
                "description": description,
                "summary": summary_data
            })

            print(f"Processed detail: {title}")
            time.sleep(1)  # 서버 부하를 막기 위한 대기 시간

        except Exception as e:
            print(f"Error processing {link}: {e}")

# 4. 데이터 저장
# Summary 데이터를 플랫하게 저장
flattened_data = []
for entry in all_data:
    for summary in entry['summary']:
        flattened_data.append({
            "Link": entry["link"],
            "Title": entry["title"],
            "Description": entry["description"],
            "Summary Title": summary["title"],
            "Summary Content": summary["content"],
        })



Processing page 0...
Processed detail: 청년 주택드림 청약통장
Processed detail: 청년월세 특별지원
Processed detail: 국민취업지원제도
Processed detail: 청년사회서비스사업단(신체건강증진서비스)
Processed detail: 청년도약계좌
Processed detail: 청년 주거급여 지원
Processed detail: 중소기업 취업청년 전월세보증금 대출
Processed detail: 청년창업 중소기업 세액 감면
Processed detail: 국민내일배움카드
Processed detail: 청년 국가기술자격시험 응시료 지원사업
Processed detail: 청년전용창업자금(창업기반 지원)
Processed detail: 전국민 마음투자 지원사업
Processed detail: 미래내일 일경험 사업
Processed detail: 서울미래인재 모집
Processed detail: 서울시 청년안심주택(공공지원민간임대) 임대보증금 지원
Processed detail: 중소기업 취업자 소득세 감면(청년)
Processed detail: 대중교통비 환급 지원 사업(K-패스)(4.24부터 카드 발급, 5.1 부터 혜택시작)
Processed detail: 청년전용 버팀목전세자금
Processed detail: 청년도전지원사업
Processed detail: 주거안정월세대출
Processed detail: 2030 무료 건강검진(청년 신체건강 인프라 확대)
Processed detail: 청년도전지원사업
Processed detail: 청년창업자금 무이자 대출지원 사업시행
Processed detail: 2024년 자립준비청년(청년 유형) 전세임대
Processed detail: 청년월세 지원사업
Processed detail: 청년 임차보증금 이자지원
Processed detail: 2024년 첫만남 이용권
Processed detail: 청년월세 한시 특별지원 2차
Processed detail

OSError: [Errno 30] Read-only file system: 'policy_details.csv'

In [3]:
print(flattened_data    )

[{'Link': 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2024011018628', 'Title': '청년 주택드림 청약통장', 'Description': '청년의 내 집 마련을 위해 장기,저리 대출 지원', 'Summary Title': '정책 번호', 'Summary Content': 'R2024011018628'}, {'Link': 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2024011018628', 'Title': '청년 주택드림 청약통장', 'Description': '청년의 내 집 마련을 위해 장기,저리 대출 지원', 'Summary Title': '정책 분야', 'Summary Content': '주거분야'}, {'Link': 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2024011018628', 'Title': '청년 주택드림 청약통장', 'Description': '청년의 내 집 마련을 위해 장기,저리 대출 지원', 'Summary Title': '지원 내용', 'Summary Content': '□ 이자율: 최대 4.5%\r□ 소득기준: 연 5,000만원(직전년도 신고소득이 있는 자 기준), 무주택자\r□ 납입한도: 월 100만원\r□ 납입금액의 40%까지 소득공제\r□ 주택청약에 당첨된 경우 청년주택드림 대출 연계\r  ㅇ 청년주택드림 청약통장 1년 이상 가입\r  ㅇ 1,000만원 이상 납입\r  ㅇ 당첨 시 분양가 80%까지 대출\r  ㅇ 금리 최저 연 2.2%(만기, 소득별 차등) 최장 40년까지 지원(고정금리)\r  ㅇ 6억 이하, 전용면적 85제곱미터 이하 주택\r  ㅇ 생애주기별 금리 인하: 결혼 0.1%p 인하, 출산: 0.5%p 인하, 다자녀: 0.2%p 인하'}, 

In [4]:
import requests
from bs4 import BeautifulSoup
import json

# 저장된 JSON 데이터 (예제)
data = [
    {'Link': 'https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2024011018628', 
     'Title': '청년 주택드림 청약통장', 
     'Description': '청년의 내 집 마련을 위해 장기,저리 대출 지원'}
]

# 결과 데이터를 저장할 리스트
detailed_data = []

# 각 링크에서 세부 정보를 크롤링
for entry in data:
    try:
        link = entry['Link']
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 모든 table_wrap 탐색
        tables = soup.find_all('div', class_='table_wrap')

        # table_wrap에서 데이터 추출
        extracted_details = []
        for table in tables:
            rows = table.find_all('li')  # 각 'li' 태그 탐색
            for row in rows:
                title = row.find('div', class_='list_tit').text.strip() if row.find('div', class_='list_tit') else None
                content = row.find('div', class_='list_cont').text.strip() if row.find('div', class_='list_cont') else None
                if title and content:
                    extracted_details.append({"Title": title, "Content": content})

        # 상세 데이터 저장
        detailed_data.append({
            "Original Link": link,
            "Policy Title": entry['Title'],
            "Description": entry['Description'],
            "Details": extracted_details
        })

        print(f"크롤링 완료: {entry['Title']}")

    except Exception as e:
        print(f"Error processing {entry['Link']}: {e}")

# 결과를 JSON 파일로 저장
with open("policy_details.json", "w", encoding="utf-8") as f:
    json.dump(detailed_data, f, ensure_ascii=False, indent=4)

print("모든 데이터가 'policy_details.json' 파일에 저장되었습니다.")


크롤링 완료: 청년 주택드림 청약통장


OSError: [Errno 30] Read-only file system: 'policy_details.json'